In [1]:
from tensorflow import keras
from keras.datasets import imdb

In [2]:
# 가장 자주 나타나는 단어 300개만 사용
(train_data, train_target), (test_data, test_target) = imdb.load_data(num_words=300)

17473536/17464789 [==============================] - 0s 0us/step


In [3]:
print(train_data.shape, train_target.shape)  # (25000,) (25000,)
print(test_data.shape, test_target.shape)  # (25000,) (25000,)

(25000,) (25000,)
(25000,) (25000,)


In [4]:
from sklearn.model_selection import train_test_split
train_data, val_data, train_target, val_target = train_test_split(train_data, train_target,
                                                                  test_size=0.2, random_state=0)

In [5]:
# 리뷰 길이를 100으로 맞추기
# 100보다 큰 경우 앞부분을 자르고, 100보다 작은 경우 앞부분에 0을 넣음
from keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_data, maxlen=100)
print(train_seq.shape)  # (20000, 100)

(20000, 100)


In [6]:
val_seq = pad_sequences(val_data, maxlen=100)
print(val_seq.shape)  # (5000, 100)

(5000, 100)


In [7]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Embedding(300, 5, input_length=100))
model.add(layers.SimpleRNN(8, return_sequences=True))
model.add(layers.TimeDistributed(layers.Dropout(0.2)))  # 각 타입 스텝별로 cost를 구해 gradient 계산
model.add(layers.SimpleRNN(8))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 5)            1500      
                                                                 
 simple_rnn (SimpleRNN)      (None, 100, 8)            112       
                                                                 
 time_distributed (TimeDistr  (None, 100, 8)           0         
 ibuted)                                                         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 8)                 136       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                        

In [12]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-simplernn.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

history = model.fit(train_seq, train_target, epochs=100,
                    validation_data=(val_seq, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
625/625 [==============================] - 23s 34ms/step - loss: 0.6941 - accuracy: 0.5197 - val_loss: 0.6465 - val_accuracy: 0.6552
Epoch 2/100
625/625 [==============================] - 21s 33ms/step - loss: 0.6029 - accuracy: 0.6835 - val_loss: 0.5987 - val_accuracy: 0.6896
Epoch 3/100
625/625 [==============================] - 23s 37ms/step - loss: 0.5553 - accuracy: 0.7341 - val_loss: 0.5396 - val_accuracy: 0.7342
Epoch 4/100
625/625 [==============================] - 21s 34ms/step - loss: 0.5372 - accuracy: 0.7426 - val_loss: 0.5293 - val_accuracy: 0.7394
Epoch 5/100
625/625 [==============================] - 22s 35ms/step - loss: 0.5283 - accuracy: 0.7510 - val_loss: 0.5297 - val_accuracy: 0.7364
Epoch 6/100
625/625 [==============================] - 22s 35ms/step - loss: 0.5175 - accuracy: 0.7564 - val_loss: 0.5223 - val_accuracy: 0.7482
Epoch 7/100
625/625 [==============================] - 21s 34ms/step - loss: 0.5135 - accuracy: 0.7604 - val_loss: 0.5240 - val_ac

In [13]:
test_seq = pad_sequences(test_data, maxlen=100)
model.evaluate(test_seq[:100], test_target[:100])

4/4 [==============================] - 0s 14ms/step - loss: 0.4861 - accuracy: 0.7700


[0.4861312210559845, 0.7699999809265137]